# Turnstile Traffic Cleaner

- This notebook reads in the weekly turnstile data published by the MTA and calculates the number of exits per turnstile per station per day
- Data can be found [here](http://web.mta.info/developers/turnstile.html)

In [3]:
import pandas as pd
import numpy as np
import sys
print sys.version

2.7.11 (default, Dec  5 2015, 14:44:47) 
[GCC 4.2.1 Compatible Apple LLVM 7.0.0 (clang-700.1.76)]


In [4]:
# read in data from mta website
data1 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_160206.txt')
data2 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_160213.txt')
data3 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_160220.txt')
data4 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_160227.txt')
data5 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_160305.txt')

# concatenate all into one big dataframe
data = pd.concat([data1, data2, data3, data4, data5])

In [5]:
data.columns

Index([u'C/A', u'UNIT', u'SCP', u'STATION', u'LINENAME', u'DIVISION', u'DATE',
       u'TIME', u'DESC', u'ENTRIES',
       u'EXITS                                                               '],
      dtype='object')

In [6]:
# fix stupid EXITS column heading
data.rename(columns = {'EXITS                                                               ':'EXITS','DATE':'DATE'}
            ,inplace = True)
data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456,BMT,01/30/2016,03:00:00,REGULAR,5519707,1863544
1,A002,R051,02-00-00,59 ST,NQR456,BMT,01/30/2016,07:00:00,REGULAR,5519718,1863554
2,A002,R051,02-00-00,59 ST,NQR456,BMT,01/30/2016,11:00:00,REGULAR,5519829,1863630
3,A002,R051,02-00-00,59 ST,NQR456,BMT,01/30/2016,15:00:00,REGULAR,5520132,1863693
4,A002,R051,02-00-00,59 ST,NQR456,BMT,01/30/2016,19:00:00,REGULAR,5520596,1863760


In [7]:
# create a unique identifier for each turnstile at a station
data['TURNSTILE'] = data['UNIT'] + data['SCP']

In [8]:
# look at stations on a given line to make sure we're spelling them right
data['STATION'][data['LINENAME']=='GL'].unique()

array(['METROPOLITAN AV'], dtype=object)

In [9]:
# create a function that reads in the station we want to see
def pick_station(station):
    station = station.upper() # convert to uppercase since that's how the data is formatted
    if station != 'METROPOLITAN AV': # there are two stations named metropolitan ave, so we handle that below
        station = data[data['STATION']==station]
    else:
        station = data[(data['STATION']==station) & (data['LINENAME']=='GL')] # only use the metropolitan ave on the GL
 
    return station

In [10]:
# change the station you want to see here:
st = 'lorimer st'

station = pick_station(st)

In [11]:
station.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,TURNSTILE
31030,H012,R268,01-00-00,LORIMER ST,LG,BMT,01/30/2016,00:00:00,REGULAR,363003,112563,R26801-00-00
31031,H012,R268,01-00-00,LORIMER ST,LG,BMT,01/30/2016,04:00:00,REGULAR,363080,112617,R26801-00-00
31032,H012,R268,01-00-00,LORIMER ST,LG,BMT,01/30/2016,08:00:00,REGULAR,363145,112630,R26801-00-00
31033,H012,R268,01-00-00,LORIMER ST,LG,BMT,01/30/2016,12:00:00,REGULAR,363503,112690,R26801-00-00
31034,H012,R268,01-00-00,LORIMER ST,LG,BMT,01/30/2016,16:00:00,REGULAR,363998,112811,R26801-00-00


In [12]:
# create lists of turnstiles and dates; these will become our column and index values
turn_list = np.asarray(station.TURNSTILE.unique())
date_list = np.asarray(station.DATE.unique())
print 'Number of turnstiles:',len(turn_list)
print turn_list
print ''
print '- - '*20
print ''
print 'Days of the week:',len(date_list)
print date_list

Number of turnstiles: 10
['R26801-00-00' 'R26801-00-01' 'R26801-06-00' 'R26801-06-01' 'R26801-06-02'
 'R35300-00-00' 'R35300-00-01' 'R35300-00-02' 'R35300-06-00' 'R35300-06-01']

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Days of the week: 35
['01/30/2016' '01/31/2016' '02/01/2016' '02/02/2016' '02/03/2016'
 '02/04/2016' '02/05/2016' '02/06/2016' '02/07/2016' '02/08/2016'
 '02/09/2016' '02/10/2016' '02/11/2016' '02/12/2016' '02/13/2016'
 '02/14/2016' '02/15/2016' '02/16/2016' '02/17/2016' '02/18/2016'
 '02/19/2016' '02/20/2016' '02/21/2016' '02/22/2016' '02/23/2016'
 '02/24/2016' '02/25/2016' '02/26/2016' '02/27/2016' '02/28/2016'
 '02/29/2016' '03/01/2016' '03/02/2016' '03/03/2016' '03/04/2016']


### This assumes that the number of people passing through a turnstile on a given day is the minimum (beginning of the day) minus the maximum (end of the day) value

In [13]:
my_list = []
for d in date_list:
    for i in turn_list:
        try:
            t_max = max(data.EXITS[(data['TURNSTILE']==i) & (data['DATE']==d)]) 
            t_min = min(data.EXITS[(data['TURNSTILE']==i) & (data['DATE']==d)])
            out = t_max - t_min
            my_list.append(out)
        except ValueError: # adding exception handling for metropolitan ave issue
            my_list.append(out)

In [14]:
# need to reshape the array since the my_list output is a Nx1 array, but we want 7xN
# use len() to account for different numbers of days and turnstiles
turn_array = np.asarray(my_list).reshape(len(date_list),len(turn_list))
turn_list = list(station.TURNSTILE.unique()) # convert to list for easier dataframe creation

In [15]:
final = pd.DataFrame(turn_array,index = date_list, columns = turn_list).T # transpose to make it more intelligible
final

,01/30/2016,01/31/2016,02/01/2016,02/02/2016,02/03/2016,02/04/2016,02/05/2016,02/06/2016,02/07/2016,02/08/2016,...,02/24/2016,02/25/2016,02/26/2016,02/27/2016,02/28/2016,02/29/2016,03/01/2016,03/02/2016,03/03/2016,03/04/2016
R26801-00-00,385,328,572,537,576,593,611,328,308,578,...,536,573,561,327,270,577,534,568,535,531
R26801-00-01,236,213,204,232,204,201,238,225,228,228,...,226,241,235,250,177,199,266,216,226,259
R26801-06-00,552,330,322,337,321,356,378,457,398,299,...,339,342,371,326,333,322,307,366,336,365
R26801-06-01,124,55,78,86,58,82,83,93,72,58,...,73,77,83,63,60,47,52,81,65,70
R26801-06-02,49,30,73,82,67,71,69,32,25,52,...,47,64,60,32,27,51,50,65,57,74
R35300-00-00,181,189,369,383,402,337,426,124,152,367,...,425,409,423,214,241,354,399,390,361,392
R35300-00-01,63,48,314,315,312,292,305,49,45,266,...,325,2576651,76,139,146,72,0,0,361,136
R35300-00-02,77,60,520,483,499,503,461,63,49,479,...,519,493,530,302,286,518,499,497,493,456
R35300-06-00,184,203,170,196,194,170,175,212,152,191,...,196,196,167,161,164,203,166,153,182,195
R35300-06-01,94,68,417,383,384,390,408,87,84,395,...,348,418,444,356,305,412,409,449,425,410


## Don't forget to change file names below
- just used this to save to the USI folder on my machine

In [16]:
#final.to_csv('USI/station_metropolitan.csv')# changed file names manually on purpose
print 'Done!'

Done!
